In [ ]:
import panel as pn
import param
from panel.reactive import ReactiveHTML

# pn.config.js_files["bodymovin"]="https://cdnjs.cloudflare.com/ajax/libs/bodymovin/5.5.3/lottie_svg.min.js"

pn.extension()

# Lottie Files using the AirBnB bodymovin js library

Lottie Files are animations you can use in your web site. They are defined in a .json file. See [lottiefiles](https://lottiefiles.com/) for more info and lots of examples. 

Airbnb provides a [.js web library](https://airbnb.io/lottie/#/web?id=html-player-installation) for playing Lottie Files that we will try to use below.

## How would users easily add .js libraries?

I tried adding

```python
pn.config.js_files["bodymovin"]="https://cdnjs.cloudflare.com/ajax/libs/bodymovin/5.5.3/lottie_svg.min.js"
```

before `pn.extension()` above. But it gives

```bash
Javascript error adding output!
Error: Mismatched anonymous define() module: function () { 'use...
```

After a lot of experimentation I find the below `require_script`workaround below. You might need to run the cell several times before it works. 

## How would users easily add their own small scripts to a component?

The `script` is not in a file. But just some script. It should only be "loaded" once. But can be used in across components and views of components

Lets define the `LottieSvg` component

In [ ]:
require_script = """
<script>
require(["https://cdnjs.cloudflare.com/ajax/libs/bodymovin/5.5.3/lottie_svg.min.js"]);
var bodymovin=require("https://cdnjs.cloudflare.com/ajax/libs/bodymovin/5.5.3/lottie_svg.min.js");
console.log(bodymovin)
</script>
"""
script = """
<script>
function createAnimation(id, path, loop, autoplay, speed){
    console.log(id, path, loop)
    let targetElement = document.getElementById(id);

    let lottieAnimation = bodymovin.loadAnimation({
            container: targetElement,
            renderer: 'svg',
            loop: loop,
            autoplay: autoplay,
            path: path,
    });
    lottieAnimation.setSpeed(speed)

    return lottieAnimation
}
</script>
"""

script_panel = pn.pane.HTML(require_script+script, width=0, height=0, margin=0, sizing_mode="fixed")

LOTTIE_FILES = [
    "https://assets5.lottiefiles.com/packages/lf20_V9t630.json",
    "https://raw.githubusercontent.com/thesvbd/Lottie-examples/master/assets/animations/skip-forward.json",
    "https://raw.githubusercontent.com/thesvbd/Lottie-examples/master/assets/animations/calendar.json",
    "https://raw.githubusercontent.com/thesvbd/Lottie-examples/master/assets/animations/loading.json",
    "https://raw.githubusercontent.com/thesvbd/Lottie-examples/master/assets/animations/menu.json",
    "https://labs.nearpod.com/bodymovin/demo/markus/halloween/markus.json",
    "https://assets5.lottiefiles.com/packages/lf20_rycdh53q.json",
]

class LottieSvg(ReactiveHTML):
    path = param.ObjectSelector(LOTTIE_FILES[0], objects=LOTTIE_FILES)
    
    plays = param.Integer()
    stops = param.Integer()
    pauses = param.Integer()
    loop = param.Boolean(False)
    autoplay = param.Boolean(False)
    
    speed = param.Integer(1, bounds=(1,10))
    
    _html = '<div class="pn-lottie" id="div-${id}"></div>'

    _scripts = {
        'render': """
            if (animation_${id}) { animation_${id}.destroy() }
            var animation_${id} = createAnimation("div-${id}", "${path}", ${loop}, ${autoplay}, ${speed})
        """,
       'plays': "if (${plays}>0) animation_${id}.play()",
       'stops': "if (${stops}>0) animation_${id}.stop()",
       'pause': "if (${pauses}>0) animation_${id}.pause()",
       'loop': "animation_${id}.loop=${loop}",
       'autoplay': "animation_${id}.autoplay=${autoplay}",
       'speed': "animation_${id}.setSpeed(${speed})",
    }
    
lottie = LottieSvg(height=400, width=400, autoplay=True)
settings = pn.Param(lottie, parameters=["path", "plays", "stops", "pauses", "loop", "autoplay", "speed", "quality"])
start_button = pn.widgets.Button(name="Start", button_type="primary")
@param.depends(start_button.param.clicks, watch=True)
def _play(*_):
    lottie.plays+=1
stop_button = pn.widgets.Button(name="Stop")
@param.depends(stop_button.param.clicks, watch=True)
def _stop(*_):
    lottie.stops+=1
start_button.on_click=_play
pause_button = pn.widgets.Button(name="Pause")
@param.depends(pause_button.param.clicks, watch=True)
def _stop(*_):
    lottie.pauses+=1
controls=pn.Column(start_button, stop_button, pause_button)
pn.Row(script_panel, pn.Column(settings, controls), lottie)

## Challenges

- Don't know how bodymovin lib should be efficiently loaded. `require_script`
- Don't know how 'utils' js code should be included. `script`
- Cannot have an action/ button on the component to play, pause or stop. That needs to be outside.

## Next Steps

- Polish into nice reference example
- Find out how I could should enable/ disable buttons